# Package Installation for Snowflake Connection and Data Processing

In [ ]:
!pip install  dask[complete]  snowflake  snowflake-connector-python snowflake-snowpark-python snowflake-snowpark-python[pandas] seaborn matplotlib numpy pandas scikit-learn  fosforml plotly

# Import necessary libraries

In [1]:
# Importing general libraries
import pandas as pd  # For data manipulation
import numpy as np  # For numerical operations
import seaborn as sns  # For data visualization
from datetime import datetime  # For date and time manipulation
import matplotlib.pyplot as plt  # For plotting graphs

# Importing Snowflake session management
from snowflake.snowpark.session import Session  # For Snowflake integration

# Importing advanced plotting libraries
from plotly.subplots import make_subplots  # For creating subplots in Plotly
import plotly.graph_objects as go  # For creating interactive visualizations with Plotly
import plotly.express as px  # For simplified plotting using Plotly

# Setting Pandas display option to show more columns in the output
pd.set_option('display.max_columns', 100)

# Importing libraries for machine learning and feature selection
from sklearn.ensemble import RandomForestClassifier  # For Random Forest model
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, precision_recall_curve, auc  # For model evaluation metrics
from sklearn.feature_selection import SelectKBest, f_classif  # For feature selection
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder  # For encoding categorical data and scaling
from sklearn.impute import SimpleImputer  # For handling missing values
from sklearn.compose import ColumnTransformer  # For applying different preprocessing steps to different columns
from sklearn.pipeline import Pipeline  # For creating machine learning pipelines
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV  # For splitting data and performing grid search for hyperparameter tuning

# Importing additional libraries for visualization and matplotlib handling
import matplotlib.pyplot as plt  # For plotting static visualizations

# Setting up environment for plotting using Seaborn and Matplotlib
sns.set(style="whitegrid")  # Setting the style for Seaborn plots


# This section connects to Snowflake using fosforml's Snowflake session manager, retrieves data from a specified Snowflake table, and loads the data into a Pandas DataFrame for further processing and modeling.

In [2]:

# Importing the get_session function from fosforml's Snowflake session manager
from fosforml.model_manager.snowflakesession import get_session

# Establishing a Snowflake session for executing queries and performing operations
my_session = get_session()

# Define the name of the Snowflake table to query
table_name = 'ORDER_DATA_TRAINING'

# Execute a SQL query to select all records from the specified table in Snowflake
df_sample = my_session.sql("select * from {}".format(table_name)).to_pandas()

# Filtering and Preparing the Training Dataset for Returned Status Analysis

In [3]:
# Import Dask for handling larger datasets efficiently
import dask.dataframe as dd

#  Dask DataFrame allows for efficient handling of large datasets and delayed execution
df_sample_dask = dd.from_pandas(df_sample, npartitions=4)  # Adjust npartitions based on data size and memory

#  We use Dask to filter the records based on the 'RETURNED_STATUS' column
df_train_dask = df_sample_dask[df_sample_dask['RETURNED_STATUS'].isin(['CANCELLED', 'RETURNED', 'DELIVERED', 'IN PROCESS'])]

#  Dask performs lazy computation, so we need to explicitly call .compute() to trigger the actual computation
df_train = df_train_dask.compute()

#  Display the count of each 'RETURNED_STATUS' to verify the filtering step
print(df_train['RETURNED_STATUS'].value_counts())


RETURNED_STATUS
CANCELLED     10000
DELIVERED     10000
IN PROCESS    10000
RETURNED      10000
Name: count, dtype: int64[pyarrow]


# Creating a Binary Target Variable for Classification

In [4]:
# If the 'RETURNED_STATUS' is 'CANCELLED' or 'RETURNED', assign 1, else assign 0
df_train['TARGET'] = df_train['RETURNED_STATUS'].apply(lambda x: 1 if x in ['CANCELLED', 'RETURNED'] else 0)


# This will give us an understanding of how many records are marked as 1 (CANCELLED/RETURNED) or 0 (DELIVERED/IN PROCESS)
target_counts = df_train['TARGET'].value_counts()

# Display the count of each target class
print(target_counts)


TARGET
1    20000
0    20000
Name: count, dtype: int64


In [5]:
df_train

,DIVISION_CODE,DIVISION_NAME,BRAND_CODE,BRAND_NAME,CLASS_CODE,CLASS_NAME,SELLING_CHANNEL,CHAIN,WEB_ORDER_NUMBER,OMS_ORDER_NUMBER,OMS_LINE_ITEM_ID,OMS_TICKET_ID,SKU_ID,QUANTITY,UNIT_PRICE,CURRENT_STATUS,CURRENT_STATUS_DESCRIPTION,TRANSACTION_DATE,SHIP_FROM_WAREHOUSE_CODE,SHIP_FROM_WAREHOUSE_DESCRIPTION,ORDER_DATE,READY_TO_PRINT_DATE,PRINT_TICKET_DATE,VERIFIED_SHIPPED_DATE,BACK_ORDERED_DATE,ORDER_AGE,GIFT_ARTICLE_FLAG,CARRIER_NAME,CARRIER_TRACKING_NUMBER,DROPSHIP_FLAG,ORDER_STATUS,ORDER_CREATION_DATE,ORDER_CONFIRMATION_DATE,WM_ORDER_ID,WM_ORDER_LINE_ID,WM_ORDER_STATUS,WM_PICKING_START_TIME,WM_PICKING_END_TIME,WM_PICKING_AGE,WM_PACKING_START_TIME,WM_PACKING_END_TIME,WM_PACKING_AGE,WM_CREATED_DATE,WM_UPDATED_DATE,WM_SHIPPED_DATE,WM_ORDER_AGE,STORE_ID,STORE_NAME,SHIP_METHOD_CODE,SHIP_METHOD_NAME,SHIP_METHOD_SERVICE,SHIPMENT_SLA,NEW_ORDER_DATE,RETURN_REASON,RETURN_FLAG,RECORD_DATE,RECORD_TIME,GROSS_SALES,RETURNED_STATUS,SHIPPING_DELAY,TARGET
0,44,Men's,10788,Boglioli,121,Suit,Store_POS,Chain1,SP0100009809137030724,51963676,51963676*1,51963676-3,301226252875,1,1275.0,Cancelled,Cancelled,2024-03-09,5,Oakbrook,2024-09-04,2024-03-08 11:10:47,2024-03-08 13:00:00,NaT,<NA>,2,false,<NA>,<NA>,false,<NA>,NaT,NaT,<NA>,<NA>,<NA>,NaT,NaT,NaN,NaT,NaT,NaN,NaT,NaT,2024-07-04,NaN,6247,NA,<NA>,<NA>,GROUND,NaN,2022-08-25,<NA>,<NA>,<NA>,<NA>,1275,CANCELLED,False,1
1,44,Men's,10864,Brunello Cucinelli,6,Denim,Others,Chain1,STA000000000541002,51890073,51890073*1,51890073-2,301230618773,1,340.0,Cancelled,Cancelled,2024-03-04,5,Oakbrook,2024-10-14,2024-03-01 20:02:26,2024-03-01 13:00:00,NaT,01-03-2024,3,false,<NA>,<NA>,false,<NA>,NaT,NaT,<NA>,<NA>,<NA>,NaT,NaT,NaN,NaT,NaT,NaN,NaT,NaT,2024-08-14,NaN,6247,NA,<NA>,<NA>,GROUND,NaN,2024-02-14,<NA>,<NA>,<NA>,<NA>,340,CANCELLED,False,1
2,56,Jewelry,45788,LIE STUDIO,36,Earrings,Online,Chain2,WC200001273540,51896750,51896750*1,51896750-2,301236804934,1,250.0,Cancelled,Cancelled,2024-03-05,3,Tampa Bay,2023-11-12,2024-03-02 13:00:00,2024-03-02 13:00:00,NaT,05-03-2024,3,false,FedEx,<NA>,false,NA,1900-01-01,1900-01-01,<NA>,******,Not in MAO,NaT,NaT,NaN,NaT,NaT,NaN,1900-01-01,1900-01-01,1900-01-01,-9999.0,6060,NA,FEDXH,FedEx Home Delivery,GROUND,NaN,2023-03-12,<NA>,<NA>,<NA>,<NA>,250,CANCELLED,False,1
3,98,Non Retail,11078,Chanel,54,Sandals,Store_POS,Chain1,SP0100003304177032024,52123853,52123853*1,52123853-2,301237490259,1,1475.0,Cancelled,Cancelled,2024-03-20,6,STORES,2023-05-05,2024-03-20 13:00:00,2024-03-20 13:00:00,NaT,20-03-2024,0,false,<NA>,<NA>,false,<NA>,NaT,NaT,<NA>,<NA>,<NA>,NaT,NaT,NaN,NaT,NaT,NaN,NaT,NaT,2023-03-05,NaN,9010,Los Angeles,PAC2,PacMan 2nd Day,2ND DAY,NaN,2022-09-05,<NA>,<NA>,<NA>,<NA>,1475,CANCELLED,False,1
4,53,Beauty,14510,ROC,995,GWP's,Online,Chain1,WC100004239069,52048953,52048953*2,52048953-2,301240889026,1,0.0,Cancelled,Cancelled,2024-03-15,3,Tampa Bay,2024-06-14,2024-03-14 13:00:00,2024-03-14 13:00:00,NaT,15-03-2024,1,true,Need it Now,<NA>,false,NA,1900-01-01,1900-01-01,<NA>,******,Not in MAO,NaT,NaT,NaN,NaT,NaT,NaN,1900-01-01,1900-01-01,1900-01-01,-9999.0,6060,NA,NDTN3,Need it Now,GROUND,NaN,2023-10-14,<NA>,<NA>,<NA>,<NA>,0,CANCELLED,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,34,Ladies Shoes,11135,Christian Louboutin,54,Sandals,Online,Chain1,WC100004207804,52001186,52001186*2,52001186-1,301233262416,-1,1195.0,Ready to Ship,Returned,2024-03-28,5,Oakbrook,2024-01-19,2024-03-12 12:26:05,2024-03-12 13:16:47,NaT,<NA>,18,false,FedEx SmartPost,******08155361239845,false,<NA>,NaT,NaT,<NA>,<NA>,<NA>,NaT,NaT,NaN,NaT,NaT,NaN,NaT,NaT,2023-11-19,NaN,6247,NA,FXPOS,FedEx SmartPost,GROUND,NaN,2023-05-19,Size Issues,<NA>,<NA>,<NA>,-1195,RETURNED,False,1
39996,11,Women's Apparel,13657,Misook,124,Dress,Online,Chain1,WC100004227187,52030192,52030192*5,52030192-2,301228013924,-1,348.0,Ready to

In [6]:
df_train['SHIPPING_DELAY']

0        False
1        False
2        False
3        False
4        False
         ...  
39995    False
39996    False
39997    False
39998    False
39999    False
Name: SHIPPING_DELAY, Length: 40000, dtype: bool

# Preparing Features and Target for Model Training

In [ ]:
# The 'TARGET' column has already been created where 'CANCELLED' and 'RETURNED' are assigned 1, and the rest are 0
# Drop 'RETURNED_STATUS' and 'TARGET' from the feature set (X_train) as they are not input features for the model
X_train = df_train.drop(columns=['RETURNED_STATUS', 'TARGET'])

# The target variable (y_train) is the 'TARGET' column we created earlier
y_train = df_train['TARGET']

# Identify the numerical and categorical columns for separate preprocessing
numerical_cols = ['SHIPMENT_SLA','UNIT_PRICE', 'WM_PICKING_AGE', 'WM_PACKING_AGE', 'WM_ORDER_AGE', 'STORE_ID',]  # Numerical features
categorical_cols = [ 'DIVISION_CODE', 'DIVISION_NAME', 'BRAND_CODE', 'BRAND_NAME', 'CLASS_CODE', 'CLASS_NAME', 'SELLING_CHANNEL', 'CHAIN', 'WEB_ORDER_NUMBER', 'OMS_LINE_ITEM_ID', 'OMS_TICKET_ID', 'SKU_ID', 'CURRENT_STATUS', 'CURRENT_STATUS_DESCRIPTION', 'SHIP_FROM_WAREHOUSE_DESCRIPTION',  'CARRIER_NAME', 'CARRIER_TRACKING_NUMBER', 'DROPSHIP_FLAG', 'ORDER_STATUS', 'WM_ORDER_ID', 'WM_ORDER_LINE_ID', 'WM_ORDER_STATUS', 'STORE_NAME', 'SHIP_METHOD_CODE', 'SHIP_METHOD_NAME', 'SHIP_METHOD_SERVICE',  'RETURN_REASON', 'RETURN_FLAG']  # Categorical features

# Display the list of numerical and categorical columns for verification
print("Numerical Columns:", numerical_cols)
print("\n\nCategorical Columns:", categorical_cols)


In [ ]:
X_train[categorical_cols] = X_train[categorical_cols].fillna(X_train[categorical_cols].mode().iloc[0])

In [ ]:
df_train[categorical_cols]

In [ ]:
# Count NaN values in each column
df_sample[categorical_cols].isna().sum()

# Preprocessing Data and Building a Machine Learning Pipeline with Random Forest Classifier

In [ ]:

#  Define a pipeline for processing numerical columns
numerical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # Impute missing numerical values with the mean
    ('scaler', StandardScaler())  # Scale the numerical features to have zero mean and unit variance
])

#  Define a pipeline for processing categorical columns
categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Impute missing categorical values with the most frequent value
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))  # One-hot encode categorical features, ignoring unknown categories
])

#  Combine both pipelines into a ColumnTransformer
# The ColumnTransformer applies the numerical and categorical transformations to their respective columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, numerical_cols),  # Apply numerical pipeline to numerical columns
        ('cat', categorical_pipeline, categorical_cols)  # Apply categorical pipeline to categorical columns
    ]
)

# The full pipeline consists of the preprocessing step followed by the RandomForest classifier
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),  # First, apply preprocessing
    ('classifier', RandomForestClassifier(random_state=42, class_weight='balanced'))  # Then, train the RandomForest model
])

pipeline

In [ ]:
for col in numerical_cols:
    mean_value = pd.to_numeric(X_train[col], errors='coerce').mean()  
    X_train[col].fillna(mean_value, inplace=True)


for col in categorical_cols:
    most_frequent = X_train[col].mode(dropna=True)[0] 
    X_train[col].fillna(most_frequent, inplace=True)

In [ ]:
X_train[categorical_cols].dtypes

In [ ]:
 X_train[col].mode(dropna=True), col


#  Hyperparameter Tuning with GridSearchCV for Random Forest Classifier

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

# This grid contains possible values for several hyperparameters of the Random Forest model.
param_dist = {
    'classifier__n_estimators': [100, 200, 300],  # Number of trees in the forest
    'classifier__max_depth': [10, 20, None],  # Maximum depth of the trees
    'classifier__min_samples_split': [2, 5, 10],  # Minimum samples required to split a node
    'classifier__min_samples_leaf': [1, 2, 4],  # Minimum samples required at each leaf node
    'classifier__max_features': ['sqrt', 'log2'],  # Number of features to consider when looking for the best split
    'classifier__max_samples': [0.5, 1.0]  # Proportion of samples to use for fitting, to avoid overfitting
}

# RandomizedSearchCV will randomly sample combinations of hyperparameters, and evaluate them.
random_search = RandomizedSearchCV(pipeline, param_distributions=param_dist, 
                                   n_iter=10, cv=3, n_jobs=-1, verbose=2, 
                                   scoring='f1_weighted', random_state=42)

# This split is necessary to evaluate model performance on unseen data (the test set).
X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Fit the model on the training data using RandomizedSearchCV to find the optimal hyperparameters.
random_search.fit(X_train_split, y_train_split)

# Step 11: Display the best hyperparameters found by RandomizedSearchCV
print("Best Hyperparameters:", random_search.best_params_)


In [ ]:
X_train_split.info()

#  Evaluating the Best Model on the Test Data

In [ ]:
# The best_pipeline contains the model with the best hyperparameters found by GridSearchCV
best_pipeline = grid_search.best_estimator_

#  Predict the labels (TARGET) for the test data and evaluate model performance
y_pred_split = best_pipeline.predict(X_test_split)

#  The classification report provides key metrics (precision, recall, F1-score) for both classes (CANCELLED/RETURNED and DELIVERED/IN PROCESS)
print("Classification Report on CANCELLED/RETURNED records:")
print(classification_report(y_test_split, y_pred_split))

#  Display the confusion matrix for further evaluation
conf_matrix = confusion_matrix(y_test_split, y_pred_split)
print("Confusion Matrix:")
print(conf_matrix)


In [ ]:
grid_search.best

#  Making Predictions on DELIVERED and IN PROCESS Orders

In [ ]:
# We only want to predict whether DELIVERED or IN PROCESS orders will be CANCELLED or RETURNED
df_predict = df_sample[df_sample['RETURNED_STATUS'].isin(['DELIVERED', 'IN PROCESS'])]

#  Drop the 'RETURNED_STATUS' column, as it's not needed for making predictions
X_predict = df_predict.drop(columns=['RETURNED_STATUS'])

#  Use the best pipeline (with preprocessing and the trained model) to predict on the filtered dataset
predictions = best_pipeline.predict(X_predict)

#  The 'PREDICTED_CANCELLED_RETURNED' column contains the predicted labels (1 for CANCELLED/RETURNED, 0 for not)
df_predict['PREDICTED_CANCELLED_RETURNED'] = predictions

#  Display the original 'RETURNED_STATUS' along with the new 'PREDICTED_CANCELLED_RETURNED' column
df_predict[['RETURNED_STATUS', 'PREDICTED_CANCELLED_RETURNED']]

In [ ]:
df_predict[['RETURNED_STATUS', 'PREDICTED_CANCELLED_RETURNED']].value_counts()

In [ ]:
# Step 11: Get the best model
best_pipeline = grid_search.best_estimator_

# Step 12: Evaluate the best model on the test split
y_pred_split = best_pipeline.predict(X_test_split)
print("Classification Report on CANCELLED/RETURNED/DELIVERED/IN PROCESS records:")
print(classification_report(y_test_split, y_pred_split))

# Step 13: Plot Confusion Matrix
conf_matrix = confusion_matrix(y_test_split, y_pred_split)

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

# Step 14: Classification Report (Precision, Recall, F1-Score) as a bar plot
report = classification_report(y_test_split, y_pred_split, output_dict=True)
report_df = pd.DataFrame(report).transpose()

plt.figure(figsize=(10, 6))
report_df[['precision', 'recall', 'f1-score']].iloc[:-3].plot(kind='bar')
plt.title('Precision, Recall, F1-Score by Class')
plt.xlabel('Class')
plt.ylabel('Score')
plt.xticks(rotation=0)
plt.ylim(0, 1)
plt.show()

# Step 15: ROC Curve and AUC (for binary classification tasks)
if len(best_pipeline.classes_) == 2:  # Check if binary classification
    # Predict probabilities for the positive class
    y_prob = best_pipeline.predict_proba(X_test_split)[:, 1]
    fpr, tpr, thresholds = roc_curve(y_test_split, y_prob)
    roc_auc = auc(fpr, tpr)

    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, label=f'AUC = {roc_auc:.2f}')
    plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate (Sensitivity)')
    plt.title('ROC Curve')
    plt.legend(loc='lower right')
    plt.show()

# Step 16: Precision-Recall Curve
precision, recall, thresholds_pr = precision_recall_curve(y_test_split, y_prob)

plt.figure(figsize=(8, 6))
plt.plot(recall, precision, label=f'AUC = {auc(recall, precision):.2f}')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc='lower left')
plt.show()

In [ ]:
df['PREDICTED_CANCELLED_RETURNED'] =  grid_search.best_estimator_.predict(df[df_sample.columns.tolist()])

In [ ]:
df[['RETURNED_STATUS', 'PREDICTED_CANCELLED_RETURNED']].value_counts()

In [ ]:
df.drop('PREDICTED_RETURN_STATUS', axis=1, inplace=True)

In [ ]:

# Importing the get_session function from fosforml's Snowflake session manager
from fosforml.model_manager.snowflakesession import get_session

# Establishing a Snowflake session for executing queries and performing operations
my_session = get_session()

# Define the name of the Snowflake table to query
table_name = 'ORDER_DATA_3009'

# Execute a SQL query to select all records from the specified table in Snowflake
df_sample = my_session.sql("select * from {}".format(table_name)).to_pandas()

df_sample

In [ ]:
df_sample[categorical_cols]

In [ ]:
df_sample.drop('PREDICTED_CANCELLED_RETURNED', axis=1, inplace=True)

In [ ]:
df_sample.info()

In [ ]:
# Convert the Pandas DataFrame (cust_df) into a Snowflake DataFrame
training_datadf = my_session.createDataFrame(df_sample)

# Write the Snowflake DataFrame to a Snowflake table named 'casino_customers'
# The 'overwrite' mode ensures that the table is replaced if it already exists
training_datadf.write.mode("overwrite").save_as_table("ORDER_DATA_FINAL")
